# Setup

In [1]:
from autogen import ConversableAgent

In [2]:
# define llm 
llm_config={"model": "gpt-3.5-turbo"}

# Creating the needed agents 

In [3]:
# agent to ask personal info (name and loc)
onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
# agent to ask the preferences topics
onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
# agent to generate info based on preference topics 
customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
# agent to act as a proxy (substitute person) for the real cust 
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS", #ask for human input 
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

# Creating tasks 

In [7]:
# list of dict that having chat interaction with different agent 
chats = [
    {
        # agent that initiating the comversation
        "sender": onboarding_personal_information_agent,
        # agent that receiving the information
        "recipient": customer_proxy_agent,
        # The message taht will be sent from the sender to the recipient 
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        # Method used for summarize
        "summary_method": "reflection_with_llm",
        # Argument for the summary method: returning customer info as a JSON object 
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        # Max number of conversational turns allowed 
        "max_turns": 2, 
        # Clear the conversation history before interaction
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

# Start the onboarding process 

In [8]:
from autogen import initiate_chats
# start the conversation 
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


d:\MINICONDA\envs\autogen_env\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to onboarding_personal_information_agent):



--------------------------------------------------------------------------------
onboarding_personal_information_agent (to customer_proxy_agent):

I'm sorry for the confusion. To better assist you, could you please provide me with your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to onboarding_personal_information_agent):

nia indonesia

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{'name': 'nia', 'location': 'indon

# Print out the summary

In [9]:
# print summary of the conversation 
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'nia', 'location': 'indonesia'}


nia is from Indonesia.


Immerse yourself in a thrilling adventure book set on a mysterious Indonesian island to make your reading experience fun. Enjoy your literary journey!




In [10]:
# print cost of the conversation 
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000161, 'gpt-3.5-turbo-0125': {'cost': 0.000161, 'prompt_tokens': 214, 'completion_tokens': 36, 'total_tokens': 250}}, 'usage_excluding_cached_inference': {'total_cost': 0.000161, 'gpt-3.5-turbo-0125': {'cost': 0.000161, 'prompt_tokens': 214, 'completion_tokens': 36, 'total_tokens': 250}}}


{'usage_including_cached_inference': {'total_cost': 4.4e-05, 'gpt-3.5-turbo-0125': {'cost': 4.4e-05, 'prompt_tokens': 73, 'completion_tokens': 5, 'total_tokens': 78}}, 'usage_excluding_cached_inference': {'total_cost': 4.4e-05, 'gpt-3.5-turbo-0125': {'cost': 4.4e-05, 'prompt_tokens': 73, 'completion_tokens': 5, 'total_tokens': 78}}}


{'usage_including_cached_inference': {'total_cost': 0.000361, 'gpt-3.5-turbo-0125': {'cost': 0.000361, 'prompt_tokens': 293, 'completion_tokens': 143, 'total_tokens': 436}}, 'usage_excluding_cached_inference': {'total_cost': 0.000361, 'gpt-3.5-turbo-0125': {'cost': 0.000361, 'prompt_tokens': 293, 'completion_tokens